# Cargar librerias

In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from dash.dependencies import Input, Output

In [4]:
JupyterDash.infer_jupyter_proxy_config()

## Cargar los datos

In [5]:
df = pd.read_csv('pidarV2.csv')
print('Estamos analizando un total de: ',len(df),'proyectos PIDAR')
df.head()

Estamos analizando un total de:  232 proyectos PIDAR


,Unnamed: 0,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,Total Víctimas,Valor cofinanciado ADR,Valor encargo fiduciario,Valor total cofinanciación ADR,Valor Contrapartida,Valor total proyecto,Hectáreas,Resolución,Fecha resolución,Vigencia,Lineamiento de consejo directivo
0,0,1,1,Implementar un sistema de ganadería silvopasto...,CAQUETA,SAN VICENTE DEL CAGUÁN,46,26,20,0,534000000,NaN,534000000,115640000,649640000,40,1220,06/29/2018 12:00:00 AM,2017,NaN
1,1,2,2,Fortalecimiento del cultivo de arroz y caña pa...,NARIÑO,SAN ANDRÉS DE TUMACO,19,7,12,0,185000000,NaN,185000000,265082460,450082460,64,1234,07/06/2017 12:00:00 AM,2017,NaN
2,2,3,3,Implementar un modelo piscícola para la produc...,CAQUETA,SAN VICENTE DEL CAGUÁN,52,36,16,31,905900513,NaN,905900513,104748820,1010649333,1,1322,08/15/2017 12:00:00 AM,2017,NaN
3,3,4,4,Mejorar las capacidades productivas para la ag...,MAGDALENA,ZONA BANANERA,150,121,29,0,2948347650,NaN,2948347650,665760350,3614108000,411,1307,08/04/2017 12:00:00 AM,2017,NaN
4,4,5,5,Mejorar las capacidades productivas de 97 fami...,CAUCA,BUENOS AIRES,97,76,21,0,500000000,NaN,500000000,581106000,1081106000,98,1215,06/29/2017 12:00:00 AM,2017,NaN


In [6]:
#Cargar Geo Json
import json
with open('Colombia.geo.json') as f:
    counties = json.load(f)

In [7]:
#Cambiar los nombre de los proyectos del df acorde con el geojson
Departamentos = []
geocol = counties.copy()
for man in geocol['features']:
    Departamentos.append(man['properties']['NOMBRE_DPT'])
    man['id']=man['properties']['NOMBRE_DPT']

In [8]:
#Nariño genera problemas por eos lo colocamos con este nombre
df.loc[df['Departamento'].str.find('NARIÑO')>=0,'Departamento']='NARIÃ‘O'

In [9]:
#Agrupar por departamtento cada una de las columnas
dff = df.groupby('Departamento').sum().reset_index()
dff.head()

,Departamento,Unnamed: 0,BP,No. VP,Total beneficiarios,Hombres,Mujeres,Total Víctimas,Valor cofinanciado ADR,Valor total cofinanciación ADR,Valor Contrapartida,Valor total proyecto,Hectáreas,Resolución,Vigencia
0,ANTIOQUIA,1733,3112,2162,1420,1068,352,387,13746800504,13773191070,21896885595,35670076665,4740,12806,34290
1,ARAUCA,330,523,397,133,82,51,47,2625468760,2625468760,1117122000,3742590760,2401,2534,6052
2,ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA...,181,349,223,90,86,4,0,755870000,755870000,299800000,1055670000,0,688,2017
3,ATLANTICO,520,879,627,571,426,145,37,6116639600,6116639600,1476559867,7593199467,212,1907,8067
4,BOLIVAR,1240,2738,1490,1406,1269,137,707,18293154276,18293154276,4662421667,22955575943,2484,4045,20166


# Codigo del dashboard

In [10]:
#Cargar la informacion e indicadores necesariso para el dash

##Cargar el logo del dashboard
import base64
test_png = './assets/my-image.png'
test_base64 = base64.b64encode(open(test_png, 'rb').read()).decode('ascii')

##Cargar los indicadores que van en la lista desplegable
available_indicators = ['Total beneficiarios', 'Hombres', 'Mujeres',
       'Total Víctimas', 'Valor cofinanciado ADR', 'Valor encargo fiduciario',
       'Valor total cofinanciación ADR', 'Valor Contrapartida',
       'Valor total proyecto', 'Hectáreas']

##Cargar la lista de años que tenemos disponible para el slider
YEARS = df['Vigencia'].unique()

In [11]:
#Definimos la funcion que cuenta el numero de proyectos del año y el recuadro qu elos despliega

bgcolor = "#f3f3f1"  # mapbox light map land color
bar_bgcolor = "#b0bec5"  # material blue-gray 200
bar_unselected_color = "#78909c"  # material blue-gray 400
bar_color = "#546e7a"  # material blue-gray 600
bar_selected_color = "#37474f" # material blue-gray 800
bar_unselected_opacity = 0.8

# Figure template
row_heights = [150, 500, 300]
template = {'layout': {'paper_bgcolor': bgcolor, 'plot_bgcolor': bgcolor}}

#Cuadro numero de proyectos
def blank_fig(height):
    """
    Build blank figure with the requested height
    """
    return {
        'data': [],
        'layout': {
            'height': height,
            'template': template,
            'xaxis': {'visible': False},
            'yaxis': {'visible': False},
        }
    }

In [13]:
# Initialize app

#Definir el servidor y el nombre del dash
app = dash.Dash(
    __name__,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

server = app.server

# App layout
app.layout = html.Div(
    id="root",
    children=[
        html.Div(
            id="header",
            children=[
                html.Img(src='data:image/png;base64,{}'.format(test_base64)),
                html.H4(children="Comprehensive Agricultural Development Plans with a Territorial Approach \
                (PIDAR for its Spanish acronym)"),
                html.P(
                    id="description",
                    children="Define which production lines should be prioritized in the selection of agricultural projects for the year 2021 and 2022 \
                    in Colombia, considering the impact generated by COVID-19 and the effects of the drop in oil. ",
                ),
            ],
        ),

        html.Div(
            id="app-container",
            children=[
                html.Div(
                    id="left-column",
                    children=[
                        html.Div(
                            id="banner",
                            children=[
                                html.Div(
                            id="slider-container",
                            children=[
                                html.P(
                                    id="slider-text",
                                    children="Drag the slider to change the year:",
                                ),
                                dcc.Slider(
                                    id="years-slider",
                                    min=min(YEARS),
                                    max=max(YEARS),
                                    value=min(YEARS),
                                    verticalHeight = 200,
                                    step=None,
                                    marks={
                                        str(year): {
                                            "label": str(year),
                                            "style": {"color": "#7fafdf"},
                                        }
                                        for year in YEARS
                                    },
                                ),
                            ], 
                            style={'width': '49%', 'padding': '0px 20px 20px 20px'}
                        ),
                        html.Div(
                    id="Number_projects",
                    children=[
                        html.P([
                            "Number of projects per year",
                        ], className="container_title"),
                        dcc.Loading(
                            dcc.Graph(
                                id='indicator-graph',
                                        figure=blank_fig(row_heights[0]),
                                        config={'displayModeBar': False},
                                        style={'width': '49%', 'padding': '0px 20px 20px 20px'}
                                    ),
                                    className='svg-container',
                                )
                            ]
                        ),
                            ]
                        ),                                
                        html.Div(
                            id="heatmap-container",
                            children=[
                                html.P(
                                    "Heatmap of PIDAR projects \
                            across the country in year {0}".format(
                                        min(YEARS)
                                    ),
                                    id="heatmap-title",
                                ),
                                dcc.Graph(
                                    id="county-choropleth",
                                    style={'width': '100%', 'display': 'inline-block', 'padding': '0 20'}
                                ),
                                dcc.Graph(id='x-time-series'),
                            ],
                        ),
                        
                    ],
                ),
                html.Div(
                    id="graph-container",
                        children=[
                        html.P(id="chart-selector", children="Select y axis indicator:"),
                        dcc.Dropdown(
                            id='yaxis_column_name',
                            options=[{'label': i, 'value': i} for i in available_indicators],
                            value='Valor cofinanciado ADR'
                        ),
                        html.P(id="charts-selector", children="Select x axis indicator:"),
                        dcc.Dropdown(
                            id='xaxis_column_name',
                            options=[{'label': i, 'value': i} for i in available_indicators],
                            value='Total beneficiarios'
                        ),
                    dcc.Graph(
                            id="selected-data",
                            figure=dict(
                                data=[dict(x=0, y=0)],
                                layout=dict(
                                    paper_bgcolor="#F4F4F8",
                                    plot_bgcolor="#F4F4F8",
                                    autofill=True,
                                    margin=dict(t=75, r=50, b=100, l=50),
                                ),
                            ),
                        ),
                    dcc.Graph(id='y-time-series'),
                    ],
                ),
            ],
        ),
    ],     
)

#Desde aca para abajo arranca el codigo o programacion del dashboard

@app.callback(
    Output("county-choropleth", "figure"),
    [Input("years-slider", "value")])
        
def update_graph(selected_year):
        filtered_df = df[df['Vigencia'] == selected_year]
        fig = px.choropleth_mapbox(filtered_df,                                      #Data
                                   locations='Departamento',                              #Column containing the identifiers used in the GeoJSON file 
                                   color='Valor cofinanciado ADR',                        #Column giving the color intensity of the region
                                   geojson=geocol,                                        #The GeoJSON file
                                   zoom=3,                                                #Zoom
                                   mapbox_style="carto-positron",                         #Mapbox style, for different maps you need a Mapbox account and a token
                                   range_color=(0, df['Valor cofinanciado ADR'].max()), 
                                   center={"lat":4.7110, "lon": -74.0721},                #Center
                                   color_continuous_scale="Viridis",                      #Color Scheme
                                   opacity=0.5,                                           #Opacity of the map
                                  )
        fig.update_layout(transition_duration=500, margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
                          height=450)
        return fig

@app.callback(Output('indicator-graph', 'figure'),    
              [Input("years-slider", "value")])

def nprojects_figure(selected_year):
    filtered_df = df[df['Vigencia'] == selected_year]
    # Count the number of selected towers
    n_selected = int(len(filtered_df))
    
    # Build indicator figure
    n_selected_indicator = {
        'data': [{
            'type': 'indicator',
            'value': n_selected,
            'number': {
                'font': {
                    'color': '#263238'
                }
            }
        }],
        'layout': {
            'template': template,
            'height': 50,
            'margin': {'l': 10, 'r': 10, 't': 10, 'b': 10}
        }
    }
    
    return n_selected_indicator

@app.callback(
    Output('x-time-series', 'figure'),
    [Input("years-slider", "value")])
def update_y_timeseries(selected_year):
    filtered_df = df[df['Vigencia'] == selected_year]
    fig3 = px.bar(filtered_df, x = 'Hectáreas', y='Departamento')
    return fig3


@app.callback(
    Output('y-time-series', 'figure'),
    [Input("years-slider", "value")])
def update_x_timeseries(selected_year):
    filtered_df = df[df['Vigencia'] == selected_year]
    fig3 = px.box(filtered_df,x="Departamento", y="Valor total proyecto")
    return fig3

@app.callback(
    Output("selected-data", "figure"),
    [Input('xaxis_column_name', "value"),
     Input('yaxis_column_name', "value"),
     Input("years-slider", "value")])
    
def update_graph2(xaxis_column_name, yaxis_column_name,selected_year):
        dff = df[df['Vigencia'] == selected_year]
        fig4 = px.scatter(x=dff[xaxis_column_name],
                     y = dff[yaxis_column_name])
        fig4.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
        fig4.update_xaxes(title=xaxis_column_name) 
        fig4.update_yaxes(title=yaxis_column_name)
        return fig4

    
#Esta ultima linea corre el dash y crea el http
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 859-519-030
Debugger PIN: 859-519-030
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
